In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import os

There are two ways to instantiate a `Model`:

1 - With the "functional API", where you start from `Input`,
you chain layer calls to specify the model's forward pass,
and finally you create your model from inputs and outputs:

```python
import tensorflow as tf

inputs = tf.keras.Input(shape=(3,))
x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
```

2 - By subclassing the `Model` class: in that case, you should define your
layers in `__init__` and you should implement the model's forward pass
in `call`.

```python
import tensorflow as tf

class MyModel(tf.keras.Model):

  def __init__(self):
    super(MyModel, self).__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)

  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(x)

model = MyModel()
```

If you subclass `Model`, you can optionally have
a `training` argument (boolean) in `call`, which you can use to specify
a different behavior in training and inference:

```python
import tensorflow as tf

class MyModel(tf.keras.Model):

  def __init__(self):
    super(MyModel, self).__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)
    self.dropout = tf.keras.layers.Dropout(0.5)

  def call(self, inputs, training=False):
    x = self.dense1(inputs)
    if training:
      x = self.dropout(x, training=training)
    return self.dense2(x)

model = MyModel()
```
File:           ~/miniconda3/envs/tensorflow2.0/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py
Type:           type
Subclasses:     DistributedCallbackModel, Sequential, LinearModel, WideDeepModel, _LinearModel, LinearModel, _DNNModel, _DNNModelV2, RNNModel

## Model

In [47]:
inputs = keras.Input(shape=(1,))
hidden1 = keras.layers.Dense(32, activation=tf.nn.relu)(inputs)
hidden2 = keras.layers.Dense(32, activation=tf.nn.relu)(hidden1)
outputs = keras.layers.Dense(1)(hidden2)

model = keras.Model(inputs=inputs, outputs=outputs)

optimizer = keras.optimizers.Adam(0.01)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                64        
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 1,153
Trainable params: 1,153
Non-trainable params: 0
_________________________________________________________________


## Checkpoint

In [9]:
print(manager.checkpoints)

['./tf_ckpts/ckpt-998', './tf_ckpts/ckpt-999', './tf_ckpts/ckpt-1000']


In [51]:
lastest_checkpoint = manager.checkpoints[-1]
ckpt.restore(lastest_checkpoint)
if lastest_checkpoint:
    print("Restored from {}".format(lastest_checkpoint))
else:
    print("Initializing from scrach.")

Restored from ./tf_ckpts/ckpt-1000


## Loss function

In [4]:
def mse(labels, preds):
    subs = labels - preds
    square = tf.math.pow(subs, 2)
    sums = tf.math.reduce_sum(square)
    mse = sums / labels.shape[0]
    return mse

In [10]:
labels = np.array([41, 45, 49, 47, 44])
preds = np.array([43.6, 44.4, 45.2, 46, 46.8])
mse(labels, preds)

<tf.Tensor: id=88624, shape=(), dtype=float64, numpy=6.079999999999994>

## Predict

In [54]:
x = np.array([30.])
model.predict(np.expand_dims(x, axis=0))

array([[59.925495]], dtype=float32)

## Train

In [8]:
# Create the metrics
loss_metric = keras.metrics.Mean(name='train_loss')

@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        predictions = model(tf.expand_dims(inputs, axis=0), training=True)
        pred_loss = mse(labels, predictions)
    
    gradients = tape.gradient(pred_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    # Update the metrics
    loss_metric.update_state(pred_loss)
    return pred_loss
    
train_data = []
for i in range(1000):
    inputs = np.array([i], dtype=np.float32)
    labels = np.array([i * 2], dtype=np.float32)
    data = (inputs, labels)
    
    train_data.append(data)
    
for epoch in range(10):
    # Reset the metrics
    loss_metric.reset_states()
    
    for inputs, labels in train_data:
        loss = train_step(inputs, labels)
        ckpt.step.assign_add(1)
        if int(ckpt.step) % 10 == 0:
            save_path = manager.save()
            print("Saved checkpoint for step {}:{}".format(int(ckpt.step), save_path))
            print("loss {:1.2f}".format(loss.numpy()))
    
    mean_loss = loss_metric.result()
    print("Epoch : ", epoch)
    print("Loss : {:.3f}".format(mean_loss))
  
# 간단하게 모델이 수렴할 수 있도록 해줌.
# model.save_weights('esay_checkpoint')

Saved checkpoint for step 10:./tf_ckpts/ckpt-1
loss 66.42
Saved checkpoint for step 20:./tf_ckpts/ckpt-2
loss 16.69
Saved checkpoint for step 30:./tf_ckpts/ckpt-3
loss 14.93
Saved checkpoint for step 40:./tf_ckpts/ckpt-4
loss 8.30
Saved checkpoint for step 50:./tf_ckpts/ckpt-5
loss 1.73
Saved checkpoint for step 60:./tf_ckpts/ckpt-6
loss 1.55
Saved checkpoint for step 70:./tf_ckpts/ckpt-7
loss 0.24
Saved checkpoint for step 80:./tf_ckpts/ckpt-8
loss 0.00
Saved checkpoint for step 90:./tf_ckpts/ckpt-9
loss 0.18
Saved checkpoint for step 100:./tf_ckpts/ckpt-10
loss 0.64
Saved checkpoint for step 110:./tf_ckpts/ckpt-11
loss 2.26
Saved checkpoint for step 120:./tf_ckpts/ckpt-12
loss 2.35
Saved checkpoint for step 130:./tf_ckpts/ckpt-13
loss 0.01
Saved checkpoint for step 140:./tf_ckpts/ckpt-14
loss 1.73
Saved checkpoint for step 150:./tf_ckpts/ckpt-15
loss 5.99
Saved checkpoint for step 160:./tf_ckpts/ckpt-16
loss 5.13
Saved checkpoint for step 170:./tf_ckpts/ckpt-17
loss 0.00
Saved checkp